$$\textbf{PLN. Tarea 2: Minería de texto básica}$$
$$\textit{Y. Sarahi García Gozález}$$

In [3]:
import re
import os
from keras.preprocessing.text import Tokenizer
from collections import Counter
import matplotlib.pylab as plt
from nltk.tokenize import TweetTokenizer
import nltk
import numpy as np
#clasificacion
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_recall_fscore_support,roc_auc_score

In [4]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version", python_version())
print("Numpy version", np.__version__)
print("NLTK version", nltk.__version__)

Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version 3.11.5
Numpy version 1.23.5
NLTK version 3.8.1


<font size=4.5 color='lightblue'>

$\textit{Ejercicio 2. Bolsas de Palabras, Bigramas y Emociones}$

$\textit{Preparación de texto, corpus y diccionarios}$

In [36]:

def get_text_from_file(path_corpus,path_truth):

    tr_txt=[]
    tr_labels=[]

    with open(path_corpus, "r") as f_corpus,open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_txt += [tweet]
        for label in f_truth:
            tr_labels += [label]   
             
    return tr_txt, tr_labels


def create_corpus_from_text(text,tokenizer):
    corpus_palabras=[]
    for documento in text:
        corpus_palabras+=tokenizer.tokenize(documento)

    return corpus_palabras


def create_dic_freq(corpus,n):
    fdist = nltk.FreqDist(corpus)
    aux=[(fdist[key],key) for key in fdist]
    aux.sort()
    aux.reverse()
    aux=aux[:n]

    return aux

def create_dic_ranking(dic_freq):
    dict_indices=dict()
    cont = 0
    for weight, word in dic_freq:
        dict_indices[word]= cont
        cont+= 1

    return dict_indices

#Clasificador

parameters={'C':[0.05,0.12,0.25,0.5,1,2,4]} #parámetro de complejidad

#llamamos el clasificador SVM
#nota: cambié el max_iter y dual porque me aparecían una serie de warnings pero no cambiaron los resultados
svr=svm.LinearSVC(class_weight='balanced',dual='auto',max_iter=3000) 

#grid search recide el objeto de clasificacion
grid=GridSearchCV(estimator=svr,param_grid=parameters,n_jobs=8,scoring="f1_macro",cv=5)



In [6]:
#guardamos los textos de entrenamiento y validación
tr_txt,tr_labels=get_text_from_file("/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_train.txt","/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_train_labels.txt")
val_txt,val_labels=get_text_from_file("/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_val.txt","/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_val_labels.txt")

In [7]:
#connvertimos a lista las etiquetas
tr_labels=list(map(int,tr_labels))
val_labels=list(map(int,val_labels))

In [8]:
#Tokenizamos
tokenizer=TweetTokenizer()
#Generamos el corpus
corpus_palabras=create_corpus_from_text(tr_txt,tokenizer)
#Generamos diccionario de frecuencias con las primeras 5 mil palabras
dict_freq=create_dic_freq(corpus_palabras,5000)
#Generamos diccionario de indices
dict_indices=create_dic_ranking(dict_freq)

In [9]:
#imprimimos algunos elementos del diccionario de indices
lista_indices = list(dict_indices.items())
for clave, valor in lista_indices[10:30]:
    print(clave, "-", valor)

me - 10
el - 11
en - 12
se - 13
es - 14
con - 15
? - 16
verga - 17
los - 18
madre - 19
por - 20
las - 21
" - 22
un - 23
te - 24
mi - 25
lo - 26
putas - 27
una - 28
... - 29


<font size=3.5 color='lightblue'>

1,2,3. Evalue BoW con pesado binario, de frecuencia y tfidf

$\textit{Bolsas de palabras con distintos equemas de pesado y sin normalizar}$

In [10]:
#________________________________PESADO_BINARIO________________________________#


#matriz_nxm de bolsa de palabras n=numero de textos, m=numero de palabras (5000)
def build_binary_bow(tr_txt,dic_freq,dic_indices):
    '''
    Función que genera la bolsa de palabras con pesado binario

    '''
    BoW=np.zeros((len(tr_txt),len(dic_freq)),dtype=int) #construimos bolsta de palabras en ceros
    cont_documento=0 #indice que recorre las FILAS

    for tr in tr_txt:#recorremos cada documento
        fdist_doc=nltk.FreqDist(tokenizer.tokenize(tr))#hacemos freqDist (tokenizado con el tokenizador ya definido) de cada documento
        for word in fdist_doc: #para cada palabra en el diccionario del documento
            if word in dic_indices: #si la palabra está en el corte final
                #AGREGAMOS un uno en el lugar correspondiente
                BoW[cont_documento,dic_indices[word]] = 1 

        cont_documento+=1

    return BoW

#________________________________PESADO_FRECUENCIA________________________________#

#matriz_nxm de bolsa de palabras n=numero de textos, m=numero de palabras (5000)
def build_freq_bow(tr_txt,dic_feq,dic_inices):
    BoW=np.zeros((len(tr_txt),len(dic_feq)),dtype=int) #construimos bolsta de palabras en ceros
    cont_documento=0 #indice que recorre las FILAS
 
    for tr in tr_txt:#recorremos cada documento
        fdist_doc=nltk.FreqDist(tokenizer.tokenize(tr))#hacemos freqDist (tokenizado con el tokenizador ya definido) de cada documento
        for word in fdist_doc: #para cada palabra en el diccionario del documento
            if word in dic_inices: #si la palabra está en el corte final
                #le asignamos su frecuencia
                BoW[cont_documento,dic_inices[word]] = fdist_doc[word]

        cont_documento+=1

    return BoW

#________________________________PESADO_TFIDF________________________________#

#primero calculamos el num de documentos en que aparece cada palabra
def df(tr_txt, dic_indices):
    df = np.zeros(5000, dtype=int)

    # comp list para las frecuencias de cada documento
    fdist_docs = [nltk.FreqDist(tokenizer.tokenize(tr)) for tr in tr_txt]

    #iteramos sobre cada palbra
    for word in dic_indices:
        #obtenemos el indice de la palabra
        index = dic_indices[word] 
        for fdist_doc in enumerate(fdist_docs): #iteramos osbre cada documento
            if word in fdist_doc: #si la palabra está en el documento actual
                df[index] += 1 #agredamos uno al conador de esa palabra

    return df

#
def build_tfidf_bow(tr_txt, dic_freq, dic_indices):
    BoW = build_freq_bow(tr_txt, dic_freq, dic_indices) #la bolsa de palabras de frecuencias es el tf
    df_list = df(tr_txt, dic_indices) #llamamos a la duncion df
    N = len(tr_txt)
    for i in range(N):
        for j, df_val in enumerate(df_list): 
            if df_val!=0: #si df es distinto de ceo (puede ser cero para el conjunto de validacion, por ejemplo que contiene menos lementos )
                BoW[i][j] *= np.log(N / df_val) #tfxdf

    return BoW

In [11]:
#generamos las bolsas de palabrascon pesado binario  de training y de validación 
binary_bow_tr=build_binary_bow(tr_txt,dict_freq,dict_indices)
binary_boW_validacion=build_binary_bow(val_txt,dict_freq,dict_indices)

In [14]:
#generamos las bolsas de palabras con pesado frecuencia de training y de validación 
frequency_bow_tr=build_freq_bow(tr_txt,dict_freq,dict_indices)
frequency_boW_validacion=build_freq_bow(val_txt,dict_freq,dict_indices)

In [27]:
#generamos las bolsas de palabras con pesado tfidf de training y de validación 
tfidf_bow_tr=build_tfidf_bow(tr_txt,dict_freq,dict_indices)
tfidf_boW_validacion=build_tfidf_bow(val_txt,dict_freq,dict_indices)

* Diferencias en bolsa de palabras

veamos la diferencia entre los pesos con cada esquema para un caso particular:

In [33]:
#el token ""@USUARIO"" corresponde al lugar 9 en el diccionario de frecuencias 
x=dict_indices.get("@USUARIO")
print("el token ""@USUARIO"" corresponde al lugar",x, "en el diccionario de frecuencias ")
print("el primer tweet de los datos (es decir el documento 0) contiene tres veces este token")
print("El peso w_(0,9) es:")
print("con pesado binario:",binary_bow_tr[0][9])
print("con pesado de frecnuencia:", frequency_bow_tr[0][9])
print("con pesado de tfidf:", tfidf_bow_tr[0][9])

el token @USUARIO corresponde al lugar 9 en el diccionario de frecuencias 
el primer tweet de los datos (es decir el documento 0) contiene tres veces este token
El peso w_(0,9) es:
con pesado binario: 1
con pesado de frecnuencia: 3
con pesado de tfidf: 5


$\textit{Clasificador}$

In [34]:
#llamamos al clasificador y le pasamos la bolsa de palabras binaria y las etiqutas
grid.fit(binary_bow_tr,tr_labels)

#predicción
y_pred_binary=grid.predict(binary_boW_validacion)

#medidas, reporte de clasificación,matriz de confusión
print(confusion_matrix(val_labels,y_pred_binary))
print(metrics.classification_report(val_labels,y_pred_binary))

[[356  62]
 [ 49 120]]
              precision    recall  f1-score   support

           0       0.88      0.85      0.87       418
           1       0.66      0.71      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.82      0.81      0.81       587



In [35]:
#llamamos al clasificador y le pasamos la bolsa de palabras de frecuencia y las etiqutas
grid.fit(frequency_bow_tr,tr_labels)

#predicción
y_pred_frequency=grid.predict(frequency_boW_validacion)

#medidas, reporte de clasificación,matriz de confusión
print(confusion_matrix(val_labels,y_pred_frequency))
print(metrics.classification_report(val_labels,y_pred_frequency))

/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/.conda/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/.conda/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[[362  56]
 [ 45 124]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       418
           1       0.69      0.73      0.71       169

    accuracy                           0.83       587
   macro avg       0.79      0.80      0.79       587
weighted avg       0.83      0.83      0.83       587



In [ ]:
#llamamos al clasificador y le pasamos la bolsa de palabras binaria y las etiqutas
grid.fit(binary_bow_tr,tr_labels)

#predicción
y_pred_binary=grid.predict(binary_boW_validacion)

#medidas, reporte de clasificación,matriz de confusión
print(confusion_matrix(val_labels,y_pred_binary))
print(metrics.classification_report(val_labels,y_pred_binary))

In [ ]:
def normalizacion_dos(X):
    norm = LA.norm(X, ord=2, axis=1)

    filas = np.shape(X)[0]
    columnas = np.shape(X)[1]

    for i in range(filas) :
        for j in range(columnas) :
            X[i][j] =  X[i][j] / norm[i]
    return X

<font size=2.5 color='lightblue'>

4,5,6. Evalue BoW con pesado binario, de frecuencia y tfidf con normalizado l2

$\textit{Bolsas de palabras con distintos equemas de pesado con normalizado L2}$

<font size=2.5 color='lightblue'>

7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.

<font size=2.5 color='lightblue'>

8. De las configuraciones anteriores elija la mejor y evalúela con más y menos términos (e.g., 1000 y 7000). Ponga una tabla dónde compare las tres configuraciones.


<font size=2.5 color='lightblue'>

9. Utilice el recurso léxico del Consejo Nacional de Investigación de Canadá llamado "EmoLex" (https://www.saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) para construir una "Bolsa de Emociones" de los Tweets de agresividad (Debe usar EmoLex en Español). Para esto, una estrategia sencilla sería enmascarar cada palabra con su emoción, y después construir la Bolsa de Emociones (BoE).


<font size=2.5 color='lightblue'>

10. Evalúa tú BoE clasificando con SVM. Ponga una tabla comparativa a modo de resumen con los tres pesados, normalize cada uno si lo cree conveniente.

<font size=4.5 color='lightblue'>

$\textit{Ejercicio 3. Recurso Línguistico de Emociones Mexicano}$

<font size=2.5 color='lightblue'>

11. Utiliceelrecursoléxicollamado"SpanishEmotionLexicon(SEL)"delDr.GrigoriSidorov, profesor del Centro de Investigación en Computación (CIC) del Instituto Politécnico Nacional (http://www.cic.ipn.mx/∼sidorov/), para enmascarar cada palabra con su emo- ción, y después construir la Bolsa de Emociones con algún pesado (e.g., binario, tf, tfidf). Proponga alguna estrategia para incorporar el "valor" del "Probability Factor of Affective use" en su representación vectorial del documento. Evalúa y escribe una tabla compara- tiva a modo de resumen con al menos tres pesados: binario, frecuencia, tfidf. Normalize cada pesado según lo crea conveniente.


<font size=2.5 color='lightblue'>

12. En un comentario aparte, discuta sobre la estrategía que utilizó para incorporar el "Probability Factor of Affective use". No más de 5 renglones.

<font size=4.5 color='lightblue'>

$\textit{Ejercicio 4. ¿Podemos mejorar con Bigramas?}$


<font size=2.5 color='lightblue'>

13. Hacer un experimento dónde concatene una buena BoW según sus experimentos anteri- ores con otra BoW construida a partir de los 1000 bigramas más frecuentes.


<font size=2.5 color='lightblue'>

14. Hacer un experimento con las Bolsas de Emociones, Bolsa de Palabras y Bolsa de Bi- gramas; usted elige las dimensionalidades. Para construir la representación final del documento utilice la concatenación de las representaciones según sus observaciones (e.g., Bolsa de Palabras + Bolsa de Bigramas + Bolsa de Sentimientos de Canadá + Bolsa de Sentimientos de Grigori), y aliméntelas a un SVM.


<font size=2.5 color='lightblue'>

15. Elabore conclusiones sobre toda esta Tarea, incluyendo observaciones, comentarios y posibles mejoras futuras. Discuta el comportamiento de la BoW de usar solo palabras a integrar bigramas, y luego a integrar todo ¿ayudó? o ¿empeoró?. Discuta también brevemente el costo computacional de los experimentos ¿Valió la Pena tener todo?. Sea breve: todo en NO más de dos párrafos.